In [9]:

# Call the scatter plot function to verify coordinates




                 player.name  location.x  location.y
33650  L. Risager Kristensen        23.0        86.0
33678              E. A´Ngon         8.0        27.0
33681         S. Dalby Wille         6.0        54.0
33684              S. Traoré        28.0        96.0
33689  L. Risager Kristensen        25.0        97.0
...                      ...         ...         ...
35375                M. Andy        83.0        58.0
35385             Z. Hussein        20.0        11.0
35392         E. Favor Fully         3.0        39.0
35406         E. Favor Fully        40.0        24.0
35412          H. Toftegaard        54.0        72.0

[101 rows x 3 columns]


2024-09-16 15:56:07.882 
  command:

    streamlit run c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()